# Imports

In [9]:
import torch
from PIL import Image, ImageOps
import pickle
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import Dataset, DataLoader
import time
import copy
import pandas as pd
import os
from functions import *

treba razdvojit cele slike i isecene
isecene pomocu modela razdvojiti na one koje se trebaju invertovati ili ne
ispitati minimum broj redova da bi se utvrdilo da li je slika cela
izracunati najmanje zastupljenu klasu
napraviti pre processing za cele slike, za isecene i za invertovanih desnih u leve i za kompletne invertovane


# Loading imige names and respective labels

In [2]:
outputs = pd.read_csv("d:/project_data/trainLabels.csv")

In [3]:
outputs.columns

Index(['image', 'level'], dtype='object')

# Inspecting number of samples for each class

In [4]:
outputs.level.value_counts()

0    25810
2     5292
1     2443
3      873
4      708
Name: level, dtype: int64

## Preparing containers for paths storage

In [8]:
num_classes = 5
img_paths_right = [ [] for _ in range (num_classes)]
img_paths_left = [ [] for _ in range (num_classes)]

proceed = 10
level_status = [0 for i in range(10)]
dir_path = 'd:\\project_data\\train_1'
image_files=os.listdir(dir_path)

## Selector checks duplicates

In [10]:
data = selector(image_files, outputs)

index = data.index
number_of_rows = len(index)
number_of_rows

35124

## Seaparting image paths for each <eye, class> based on lowest class distribution

In [11]:
row = 0

maxi = 350 # number of samples per class per eye

while proceed:
    try:
        #print(data["level"].value_counts())
        #print(len(image_files)
        level = int(data["level"][row])
        
        abs_path = os.path.join(dir_path, data['image'][row]) + '.jpeg'
        if 'left' in abs_path:
            if level_status[level] < maxi:
                img_paths_left[level].append(abs_path)
                level_status[level]+=1 
            elif level_status[level] == maxi:
                proceed-=1
                level_status[level]+=1            
        elif 'right' in abs_path:
            if level_status[level+5] < maxi:
                img_paths_right[level].append(abs_path)
                level_status[level+5]+=1   
            elif level_status[level+5] == maxi:
                proceed-=1
                level_status[level+5]+=1                        
    except:
        pass
    
    if row%100==0:
        print(f'{row}/{maxi*num_classes*2}') 
        print(level_status)
    row+=1
    if row > number_of_rows:
        proceed = 0

0/3500
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
100/3500
[40, 3, 6, 1, 1, 41, 1, 6, 1, 1]
200/3500
[77, 8, 12, 2, 2, 79, 4, 13, 2, 2]
300/3500
[114, 10, 20, 3, 4, 114, 7, 22, 3, 4]
400/3500
[149, 13, 29, 3, 7, 148, 11, 28, 5, 8]
500/3500
[184, 17, 35, 7, 7, 181, 16, 37, 6, 9]
600/3500
[224, 18, 41, 9, 8, 220, 18, 44, 7, 10]
700/3500
[266, 19, 48, 9, 8, 265, 20, 47, 7, 10]
800/3500
[300, 21, 60, 10, 9, 302, 23, 55, 8, 11]
900/3500
[333, 27, 67, 12, 11, 337, 27, 64, 9, 12]
1000/3500
[351, 31, 73, 16, 12, 351, 28, 75, 12, 14]
1100/3500
[351, 37, 77, 16, 13, 351, 30, 80, 13, 14]
1200/3500
[351, 40, 86, 19, 14, 351, 32, 88, 15, 15]
1300/3500
[351, 44, 101, 19, 14, 351, 36, 99, 15, 15]
1400/3500
[351, 46, 108, 21, 15, 351, 39, 107, 16, 16]
1500/3500
[351, 51, 111, 21, 15, 351, 45, 114, 16, 16]
1600/3500
[351, 54, 123, 22, 16, 351, 48, 127, 16, 16]
1700/3500
[351, 57, 132, 22, 17, 351, 51, 136, 17, 17]
1800/3500
[351, 61, 135, 22, 17, 351, 53, 140, 18, 17]
1900/3500
[351, 62, 139, 23, 18, 351, 57, 147,

## Save paths in files

In [ ]:
import pickle

lefts = f'nochers_5x{maxi}_left'
rights = f'nochers_5x{maxi}_right'

with open(lefts, 'wb') as f:
    pickle.dump(img_paths_left, f)
with open(rights, 'wb') as f:
    pickle.dump(img_paths_right, f)